In [1]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
# To print the tree
from sklearn.tree import export_graphviz

In [2]:
iris=datasets.load_iris()
x_train,x_test,y_train,y_test= train_test_split(iris.data,iris.target,random_state=1)

In [3]:
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [4]:
import pydotplus
dot_data=export_graphviz(clf,out_file=None,
                        feature_names=iris.feature_names,
                        class_names=iris.target_names)
graph=pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("iris.pdf")

True

In [5]:
y_train_pred = clf.predict(x_train)
y_test_pred = clf.predict(x_test)

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
confusion_matrix(y_train,y_train_pred)
# all entries are correct, there may be overfitting

array([[37,  0,  0],
       [ 0, 34,  0],
       [ 0,  0, 41]], dtype=int64)

In [8]:
confusion_matrix(y_test,y_test_pred)

array([[13,  0,  0],
       [ 0, 15,  1],
       [ 0,  0,  9]], dtype=int64)

In [9]:
# CODE

In [10]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
iris=datasets.load_iris()
x_train,x_test,y_train,y_test= train_test_split(iris.data,iris.target,random_state=1)

In [11]:

class TreeNode:
    def __init__(self,level):
        self.__level=level
        self.splittingFeature=""
        self.__entropy=0.0
        self.__gainRatio=-1
        self.childList=[]
        self.list=[]
        self.result=""
    
#     def getLevel():
#         return self.__level
    def addChild(self,node,data):
        self.list.append(node)
        
        self.childList=self.childList+[(node,data)]
    
    
    def setSplittingFeature(self,f):
        self.splitting_feature=f
    
    def printCount(self,y):
        # this method is used for printing the unique value (with their count)
        # present in the output i.e. Y
#         y = np.array(y)
#         y = pd.Series(y)
        # converting y to pandas'series and using value_counts() to get the unique value
        a=y.value_counts(ascending=True)
        
        # printing the unique values of Y
        for i in range(a.shape[0]):
            print("Count of ",i,"(",str(a.index[i])+")",a.values[i])
        self.result=str(a.index[a.shape[0]-1])
            
    def setEntropy(self,entropy):
        # method to add entropy as the fields
        self.__entropy=entropy
    
    def setGainRatio(self,gainRatio):
        self.__gainRatio=gainRatio
    
    def printDetails(self,y):
        print("Level",self.__level)
        self.printCount(y)
        print("Current Entropy  is =",self.__entropy)
        
        if(self.splittingFeature!=""):
            
            print("Splitting on feature",self.splittingFeature,"with gain ratio",self.__gainRatio)
        else:
            print("Reached leaf Node")
        print("result",self.result)

In [12]:
import math

def gain(x, y, f):
    #Using gain ratio for calculation
    labels=getLabels(x[f])
    x["y"]=y
    
    # calculating the split info
#     print(";;;;;",f)
#     print(x)
#     print(x[f])
    
    split_info_ = split_info(x[f])
    
    a=x["y"].value_counts()
    length_y=x.shape[0]
    
    info_feature=info(a,length_y)
    
    
#     print("aaaaaaaaaaaaaaaaaaaaa")
#     print(a)
#     print(x)

#     print(info_feature)
    
    
    # Calculating the entropy/ Info gain
    entropy=0
    for i in labels:
        new_x=x[x[f]==i]
        a1=new_x["y"].value_counts()
        length_y1=new_x.shape[0]
        
        info_i=info(a1,length_y1)
        
        entropy+=float(new_x.shape[0]*info_i)/float(x.shape[0])
        
    info_gain=info_feature-entropy
    
    gain_ratio=info_gain/split_info_
#     print("infogain",info_gain,"a-b; a",info_feature,"b",entropy,"split_info",split_info_,"gain ratio",gain_ratio)
    return (info_feature,gain_ratio)
    
    
def split_info(x):
    split_info=0
    a=x.value_counts()    
    for i in range(a.shape[0]):
        p=float(a.values[i])/float(x.shape[0])
        log=math.log(p,2)
        split_info += p*log
    split_info= (-1)*split_info
    return split_info

def info(a,length_y):

    info_i=0
        
    for i in range(a.shape[0]):
        p=float(a.values[i])/float(length_y)
        log=math.log(p,2)
        info_i += p*log
#         print(i,p,log,info,"-------------")
    info_i= (-1)*info_i
    
    return info_i
    
def getLabels(x):   
    # x is pandas' series
    
    a=x.value_counts()
    length=a.shape[0]
    
    labels=[]
    for i in range(a.shape[0]):
        labels.append(a.index[i])
    return labels

def decisionTree(x, y, level, features):
    print()
    # Creating the Tree Node object 
    node=TreeNode(level)
    
    # creating set for finding the unique values of y
    a=set()
    for i in range(y.shape[0]):
        a.add(y.iloc[i][0])
    
    if(len(a)==1):
        # pure node is achieved
        
        node.printDetails(y)
        
        return node
    
    elif(len(features)<=0):
        # no more features to split upon
        a=y.value_counts()
        length_y=x.shape[0]

        info_feature=info(a,length_y)
        node.setEntropy(float(info_feature))
        node.printDetails(y)
        return node
    
    # Finding the best feature to split upon
    max_gain=-1
    max_entropy=-1
    best_feature=""
    
    for f in features:
        # calculating gain ratio
        g1=gain(x, y, f)
        g=g1[1]
        entropy=g1[0]
        # updating the best feature according to the maximum gain
        if(g>max_gain):
            best_feature=f
            max_gain=g
            max_entropy=entropy
    
    # removing the current feature
    features.remove(best_feature)
    
    # printing the current Node
    node.setSplittingFeature(best_feature)
    node.splittingFeature=best_feature
    node.setEntropy(max_entropy)
    node.setGainRatio(max_gain)
    node.printDetails(y)
    
    # Recursion to build the complete tree
    
    labels=getLabels(x[best_feature])
    x["y"]=y
    for i in labels:
        
        new_x = x[x[best_feature]==i]
        new_y = new_x["y"]
        new_x.drop('y',axis=1,inplace=True)
        
        # recursion
        node.addChild(decisionTree(new_x, new_y, level+1, features),i)
    
    x.drop('y',axis=1,inplace=True)
    return node
           

In [13]:
X=[["True","True"],["True","False"],["False","True"],["False","False"]]
# Y=[["True"],["True"],["True"],["False"]]
Y=["True","True","True","False"]

X=np.array(X)
Y=np.array(Y)
x = pd.DataFrame(X,columns=["x1","x2"])
y = pd.Series(Y)
node=decisionTree(x, y, 0, ["x1","x2"])
# for i in range(y.shape[0]):
#     print(y.iloc[i][0])
# y.shape
# a=y.value_counts()
# print(a)
# a.shape()



Level 0
Count of  0 ( False) 1
Count of  1 ( True) 3
Current Entropy  is = 0.8112781244591328
Splitting on feature x1 with gain ratio 0.31127812445913283
result True

Level 1
Count of  0 ( True) 2
Current Entropy  is = 0.0
Reached leaf Node
result True

Level 1
Count of  0 ( False) 1
Count of  1 ( True) 1
Current Entropy  is = 1.0
Splitting on feature x2 with gain ratio 1.0
result True

Level 2
Count of  0 ( True) 1
Current Entropy  is = 0.0
Reached leaf Node
result True

Level 2
Count of  0 ( False) 1
Current Entropy  is = 0.0
Reached leaf Node
result False


C:\Users\MOHIT\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-12-028233cdb422>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["y"]=y
<ipython-input-12-028233cdb422>:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["y"]=y


In [14]:
def fit(x,y,features):
    decisionTree(x, y, 0, features)
def predict_single(node,x):
    print("--------")
    print("level",node._TreeNode__level,"result",node.result)
#     print("selected feature",node.splitting_feature,"level",node._TreeNode__level,"result",node.result)
  
    print(node.result,len(node.childList))
    if (len(node.childList)==0):
        return node.result
    x_data=x[node.splittingFeature]
    x_data=x_data.iloc[0]
    for i in range(len(node.childList)):
        value=node.childList[i]
        curr_node=value[0]
        curr_node_data=value[1]
#         print("passing",curr_node,curr_node_data)
            
        if(x_data==curr_node_data):
            return predict_single(curr_node,x)
    print("-------------------------------------------------------------------------")
def predict(node,x):
#     print(x.shape)
    y=[]
    for i in range(x.shape[0]):
        a=x.iloc[i:]
#         print("a",a.shape)
#         print(a)
        result=predict_single(node,a)
        print("------------------result----------",result)
        y.append(result)
    Y=np.array(y)
    y = pd.Series(Y)
    return y
        
def score(y_pred,y_true):
    count=0
    for i in range(y_true.shape[0]):
        a=y_true.iloc[i]
        b=y_pred.iloc[i]
        
        if(a==b):
            count+=1
    score=count/y_true.shape[0]
    return score
    

In [15]:
# print(type(node.childList))
# print(len(node.childList))
# print(node.splittingFeature)
# X=[["False","True"]]
# x = pd.DataFrame(X,columns=["x1","x2"])
# print(x)
# print(x.shape)
from sklearn.metrics import confusion_matrix
X=[["True","True"],["True","False"],["False","True"],["False","False"]]
X=np.array(X)
x = pd.DataFrame(X,columns=["x1","x2"])
# print(x)
y_pred=predict(node,x)
print("SCORE",score(y_pred,y))

--------
level 0 result True
True 2
--------
level 1 result True
True 0
------------------result---------- True
--------
level 0 result True
True 2
--------
level 1 result True
True 0
------------------result---------- True
--------
level 0 result True
True 2
--------
level 1 result True
True 2
--------
level 2 result True
True 0
------------------result---------- True
--------
level 0 result True
True 2
--------
level 1 result True
True 2
--------
level 2 result False
False 0
------------------result---------- False
SCORE 1.0


In [16]:
import numpy as np
np_array=np.array([1,2,3,4,"Aba","aca"])
import pandas as pd
new_series = pd.Series(np_array)
a=new_series.value_counts()
try:
   val = int(input())
    
except ValueError:
   print("That's not an int!")
print(type(int(a.index[0])),"----",a.index[0])
for i in range(a.shape[0]):
    print(a.index[i],a.values[i])
# print(new_series)

asc
That's not an int!
<class 'int'> ---- 4
4 1
3 1
2 1
Aba 1
aca 1
1 1


In [17]:
a=["ab","cd"]
a.remove("ab")
a

['cd']

In [18]:
import math
math.log(10,math.e)

2.302585092994046

In [19]:
a=[1,2,3]
b=a+[2]
b

[1, 2, 3, 2]